# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
# 提示：可參考以下的Stack Overflow: 
# https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166

browser = webdriver.Chrome()
browser.get(ETtoday_url)

SCROLL_PAUSE_TIME = 1

# 用Selenium模擬下拉網頁動作，讓網頁更新
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)

----------------------------------------------------------------------

[0] 「境界」電腦新系統仍須時間改善　南山人壽、南山產險遭降信評

▲南山人壽。（圖／記者湯興漢攝）
記者吳靜君／台北報導
中華信評今（21）日表示，系統改善仍需時間解決，南山人壽與南山產物評等調降為「twAA」;信用觀察狀態解除，展望轉為「穩定」。
南山人壽遭降評之前的評等長期發行信用評等為「twAA+」、10月時因為境界系統問題，引發保戶的權益受損，遭金管會處分停止販售投資型保單，評等展望列入「觀察負向」。
----------------------------------------------------------------------

[1] 商業周刊／星宇小年夜首航！　一碗熱濃湯揭張國煒殺出重圍戰法

▲張國煒的首架飛機，以大地暖色系打造家的舒適感。（圖／攝影程思迪）
文／韓化宇
籌備3年之久的星宇航空，首個航班於1月23日上午7點25分飛往澳門，首航儀式結束後，創辦人張國煒親自駕駛第二個飛往澳門的航班，宣告台灣航空市場雙雄割據的局面，自此結束。
這次，搭乘首航的顧客，將享受到怎麼樣的服務？透過以下細節，可以看到一個後進者，很不一樣的策略抉擇。
經濟艙遞馬鈴薯濃湯
航空公司通常1到2個月更換一次菜色，而星宇公布的首波飛機餐，經濟艙會提供每個客人一碗馬鈴薯濃湯。
經濟艙附熱湯，在國際上較知名的航空公司，像日本航空曾和有「傳說中沾麵」之稱的大勝軒合作，推出用冷水浸泡過的麵，浸入熱湯的「溫麵」，頗受好評。
這是華航、長榮不做的服務。因為，一碗小小的熱湯，會為航空公司的服務流程增添很多麻煩。
例如，客人不小心打翻弄髒地毯或座椅，不像打翻熱茶容易清洗，最糟的情況，可能是要換掉整塊地毯。此外，把上百份的熱湯搬上飛機，也會增加額外的重量，以及耗油量，這也是成本。
星宇負責客艙服務品的主管表示，人的味覺在高空上會疲乏，鹹的食物具有刺激味蕾之效，所以，在飛機上吃泡麵，總覺得特別美味，而一碗熱湯，不僅暖胃、也暖心。
透過小服務，創造客人最大的心理滿足，這是很好的投資。
恢復消失已久的頭等艙
星宇預計2021年引進雙走道客機A350，屆時將設置在國內銷聲匿跡已久的頭等艙，而且，會配置4個艙等，即頭等艙、商務艙、豪華經濟艙、經

▲AI職缺是5年前的3.2倍，半導體平均年薪86.6萬。（圖／pixabay）
記者余弦妙／台北報導
年後想跳槽，這些產業年薪最高，根據調查2019年薪最高為半導體產業，可達86.6萬元，其次為電腦及消費性電子製造業82萬元、倉儲或運輸輔助業80.6萬。
104人力銀行總經理黃于純表示，年前線上工作數已達64.6萬個，民生消費產業為人才需求大宗，電子業利多頻傳熱度不墜，其中，AI、5G、電商、懶人宅經濟大爆發，AI相關職缺已是5年前的3.2倍。
104人力銀行指出，傳統1月份屬於年度的徵才淡季，但企業搶才鬆懈不得，2020年1月仍有超過3萬家企業在104人力銀行開出64.6萬個工作機會。
對此，黃于純說，產業薪資反映過去營收、目前經營、以及未來願景，根據104《薪酬白皮書》，平均年薪最高的前十大產業，多和半導體、ICT資通訊、軟體網路、以及金融相關，其中，AI和5G題材涉及層面廣泛，半導體平均年薪86.6萬元居各產業之首。
分析104人力銀行資料庫，2014年平均每月有2,468個AI相關的工作機會，截至2019年已達7,926個，是五年前的3.2倍。人才需求集中於半導體、ICT資通訊產業的工程師。
AI相關人才需求前十職缺中就七個為工程師，當中由軟體設計工程師奪冠，其次為演算法開發工程師、韌體設計工程師。除工程師技術職之外，國內業務人員的需求也日趨增加。
----------------------------------------------------------------------

[12] 前陣子夢到自己刮出頭獎！　桃園男果真刮中頭獎200萬

▲過年期間深受民眾喜愛的刮刮樂，台彩今年推出17款刮刮樂，與大家一起歡度金鼠年。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
一年一度的農曆春節即將到來，不少民眾迫不及待買張刮刮樂試手氣，全台各地陸續傳出刮中大獎的好消息。台彩今（21）日透露，一名年約50歲的桃園男熟客，前陣子夢見自己刮出頭獎，沒想到如今那場夢竟然成真，幸運刮出頭獎200萬元，期許好運能一直持續下去。
台彩今年初推出的「金鼠報喜」，每張售價200元，其中頭獎200萬元有6個，而1,000元（含）以上的獎項超過23萬個，總中獎率35%。這次財神爺眷顧桃園市觀音區的凱發商行，刮出大獎的是一位年紀約50幾歲的男性常客。
根據凱發商行店員描述，這名男

▲OnePlus下一代旗艦機計畫使用120 Hz螢幕。（圖／翻攝自OnePlus官網）
記者謝仁傑／綜合報導
OnePlus正計畫將更高的120 Hz螢幕更新率AMOLED顯示螢幕打到即將推出的OnePlus 8 Pro中，比起目前主流的60Hz多上一倍，業界評論，等於是到電競等級手機。
根據之前的渲染圖顯示，OnePlus 8系列將捨棄彈出式鏡頭，使用前螢幕的穿孔式鏡頭。
現在手機的硬體規格越來越高，甚至造成效能過剩的問題，因此廠商開始向提升多媒體的使用體驗前進，創造更具吸引力的產品。例如相機的畫素越來越高，甚至許多新機都標榜使用90Hz螢幕更新率，螢幕更新率越高，使用者的觸控體驗會更好。
----------------------------------------------------------------------

[24] 2020年美國經濟維持增長　股債投資各有兩大方向值得關注

▲ 保德信投信預期2020年美股表現可以期待。（圖／路透）
記者李瑞瑾／綜合報導
回首2019年，是市場振奮的一年，科技股屢創新高，一掃2018年貿易戰重挫陰霾。展望2020，激情過後將回歸基本面，穩增長成為今年市場主旋律，貨幣寬鬆政策持續，美國就業強勁帶動消費動能，惟基期因素先關注油價，通膨數據將會影響政策判斷，保德信投信建議，投資人要審慎分散投資，賺取低風險市場利潤，透過穩健標的提高回報預期。
去年美國聯準會三度預防性降息，聯準會主席鮑爾對於美國經濟前景感到滿意，觀察美國聯準會會議紀錄來看，聯準會認為目前利率政策足夠應付市場現況。再則，美國非農就業數字月平均位於15萬人之上，失業率維持低檔，良好就業數據除增添消費信心及薪資成長外，家庭負債比率降低，代表人民可支配所得增加，美國消費力道，支撐經濟成長，更成為經濟增長重要引擎之一。
▲ 美國經濟數據持穩，支撐經濟成長動能。（圖／路透社）
----------------------------------------------------------------------

[25] 國泰置地廣場桃園今夏開幕！青埔建設再添利多　自住客大舉回流

圖、文／富比士提供
甩脫鬼城名號，桃園青埔特區近年夯到爆！重大建設集中火力亮相，除了既有的華泰名品城I、II、III期，2020年最讓人期待的將是「國泰置地廣場‧桃園」，囊

圖文／鏡週刊
 
近日 PS4 大特價，市場上關於 PS5 的傳言總不見少，本篇整理遊戲論壇和媒體對 PS5 的最新臆測。
 
不只是遊戲可以在新主機上運行，另一名匿名爆料者「PS5 “secret” information Anonymous」描述了更多移植細節，他稱，玩家在 PS4 的遊戲進度、角色和備份資料都可以無痛轉移到 PS5，據說 PS5 還支援 PS4 現有配件，堪稱擁有最完美的兼容性。
去年便有爆料指出 Sony 預計在 2 月 12 日的 PlayStation Meeting 上正式讓 PS5 亮相，不過近日英美論壇 4 chan 有消息道，Sony 可能提前在 2 月 5 日，紐約的 Sony Hall 讓 PS5 出場，論壇網友回報目擊多名 Sony 高層出現在舊金山的消息，增加爆料的可信度。
連 PS5 的首波遊戲名單都被爆料，除了先前已經確認過的獨佔遊戲《Godfall（神之隕落）》，還有《跑車浪漫旅 7（Gran Turismo 7）》、《MLB The Show 21》、曾經是 PS3 獨佔遊戲的《惡魔靈魂 重製版》，以及從《戰神（God of War）》系列延伸出的全新遊戲《Legendz》。
最後，PS5 的發售時間從 10 月到 12 月都有人說，猜測定價則差不多都落在 1.5 萬左右。
►「2020愛妳愛妳」～超殺新年禮盒來了！
----------------------------------------------------------------------

[37] 台商笑了！萬坪廠區停工15年　意外爆賺26倍！

圖文／鏡週刊
面對中美貿易戰，群光董事長許崑泰化危機為轉機，去年8月決定重啟閒置15年的泰國廠，並在旁購地11000坪蓋新廠。但其實，群光閒置15年的廠，竟意外爆賺26倍。
 
「33年前，群光是全世界第一個到泰國設廠的電腦周邊零組件廠商。未來產能配置則看是否（被美國）課到稅，我們會根據客戶的選擇調整，中國產能目前沒有縮減。」群光董事長許崑泰笑著說，泰國廠如果全能量產，產能將比現在多出兩成，他信心滿滿的說：「今年群光營收將突破1000億元。」
講到泰國廠，許崑泰有滿滿的情感。原來許崑泰1983年成立群光，3年後就到泰國設廠，爾後中國供應鏈崛起，群光也跟著客戶腳步到中國設廠，泰國廠停工後只留下3個員工看

▲中東局勢不穩定，影響全球油價。（圖／路透社）
記者謝仁傑／綜合報導
利比亞的石油生產幾乎停止，他們的武裝部隊控制了港口的進出，石油產量受到影響；同時，石油輸出國組織（OPEC）第二大產油國伊拉克，也面臨石油供應的威脅。
今年從美國與伊朗的緊張局勢，到利比亞的動亂以及伊拉克主要油田的生產停止。雖然非OPEC成員國的原油產量可能激增，將會限制油價進一步上漲，但市場目前仍將繼續受到這些地緣政治風險的影響。
----------------------------------------------------------------------

[48] 鴻海去年發出1795萬照顧484位寶寶　今年愛心嘉年華贊助百家社福團體

▲鴻海去年的愛心嘉年華。（圖／記者李毓康攝）
記者姚惠茹／台北報導
鴻海（2317）愛心嘉年華22日將在南港展覽館盛大登場，今年除了備受關注的藝人卡司名單，鴻海也舉辦「愛心市集」，贊助近百家社福團體免費設攤，更首度攜手兒童福利聯盟，推出春節親子手作DIY與員工親子同樂活動，並宣布在去年為照顧484位0～3歲的「鴻海寶寶」發出1795萬津貼。
鴻海表示，今年愛心嘉年華會，為了歲末送暖，攜手兒福規劃一系列親子活動，包括有「小小孩的唱跳派對」、「捧心娃故事屋」、「愛的故事屋」；春節親子手作DIY部分有「開運擴香DIY」、「賀新春紅包袋」、「春聯創作」及「迎春攝影棚」等，目前已吸引近千位鴻海同仁眷屬搶先報名。
鴻海指出，集團從2015年起，開創全台灣企業先例，推行全方位的「準媽媽照護計畫（媽媽福袋）」，計畫至今已滿五年，並在2019年度集團的生育及養育津貼照護網，總共有484位0～3歲的「鴻海寶寶」受惠，發出的總津貼高達1795萬元。
鴻海補充，媽媽福袋計畫按懷孕前、中到產後，給予準媽媽們全方位補助，且體諒女性員工懷孕期間舟車勞頓，預產期前兩個月，每月給予1.5萬車資補助，讓孕媽咪們能利用計程車代步，務求懷孕員工出行安全第一。
鴻海說明，除了0～3歲間生養育津貼外，鴻海也與何嘉仁、長頸鹿、培幼文教等全台158所幼兒園合作「企業托育」，給予「鴻海寶寶」們入學優惠。
此外，2020鴻海愛心嘉年華中，也展出「永齡．鴻海台灣希望小學」、「鴻海獎學鯨」、「永齡基金會」等社會公益關懷成果，今年的國歌除了按往例由「永齡．鴻海台灣希望小學」小學生擔任領唱團外，也邀請歷

▲經濟部表示，在半導體業者先進製程具競爭優勢下，有助外銷接單回穩，今年第一季外銷訂單表現可望轉正。圖為經濟部統計處長黃于玲。（圖／記者林淑慧攝）
記者林淑慧／台北報導
經濟部統計處今（20）日公佈2019年12月外銷訂單統計，訂單金額為437.8億美元，經季節調整後年增2.9％，結束連續13個月負成長走勢；經濟部統計處長黃于玲表示，半導體業者先進製程具競爭優勢下，有助外銷接單回穩，今年第一季外銷訂單表現可望轉正。
黃于玲指出，2018年12月受惠傳統貨品隨著春節備貨需求啟動，接單普遍較上月回升，另外5G通訊及高效能運算需求成長，加上比較基期相對較低，傳統貨品減幅收斂，推升12月外銷訂單年增0.9%，為2018年11月以來首次轉正。
依據經濟部統計，受到中美貿易紛爭干擾，全球貿易保護主義興起及國際原材物料下跌影響，傳統貨品接單明顯衰退，科技類產品亦因智慧手機漸趨飽和、筆電、伺服器接單減緩，加上面板市場供過於求而價格走跌，2019年全年外銷訂單累計4845.6億美元，較上年同期減少272.6億美元，年減5.3%，全年接單金額確定負成長。
按貨品別觀察，資訊通信及電子產品因智慧手機、5G通訊等接單成長，分別年增0.4%及4.7%；光學器材因中國大陸面板產能開出，市場供過於求，面板價格仍低，年減8.8%，惟減幅已縮小。
傳統貨品方面，因全球景氣仍顯疲弱，市場需求減少，加上產品價格仍低於上年同期，致基本金屬年減6.5%、塑橡膠製品年減5.2%、化學品年減8.1%；惟機械因中美貿易紛爭情勢緩和，漸次恢復投資動能，較去年同期成長5.1%，為連續13個月負成長後首次轉正。
展望未來，黃于玲指出，第一季為電子產業淡季，加上全球經濟成長平疲，需求仍緩，恐抑制接單成長動能，惟中美貿易緊張情勢趨於緩和，國際油價、鋼價回升，有助緩解市場觀望氣氛,加上5G 通訊、人工智慧、高效能運算、物聯網等新興科技需求持續擴增，可望挹注外銷訂單接單動能升溫。
----------------------------------------------------------------------

[59] 金豬年台股市值大增6.86兆元　平均每位股民可賺103萬元！

▲ 台股豬年封關日收紅。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
台股今（20）天以12,118點封關，農曆豬年全年上漲了22%；上市

▲彰化年輕夫妻散步途中聽見鸚鵡高喊「中獎」，決定買刮刮樂試手氣，幸運刮中100萬元大獎。（圖／台彩提供）
記者紀佳妘／台北報導
台彩推出「2000萬超級紅包」刮刮樂，上周已被刮出一個三獎100萬元，今（20）日再度傳出好消息，有一對彰化年輕夫婦散步途中，聽到捷利農產商行門外的鸚鵡高喊「中獎」，決定買刮刮樂試手氣，沒想到幸運刮中100萬元，兩人抱在一起大叫，直呼「有買有希望」。
台彩推出「2000萬超級紅包」短短的時間，全台各地陸續傳出刮出大獎的好消息。這次是彰化縣彰化市捷利農產商行傳出捷報，刮出三獎100萬的是一對年約30幾歲的熟客夫妻。
台彩表示，這對夫婦當天只是散步經過店門口，原先並沒有要進店裡購買的，不過由於店外有隻羽毛非常漂亮的鸚鵡，之前有耳聞過他們家的鸚鵡是會講話的，中獎當天這對夫妻就是看到這隻鸚鵡一直很激動的對他們說，「你好！你好！中獎！中獎！」。
台彩指出，這一對夫妻心想來過這麼多次，都沒有聽過牠講話，當天卻這麼難得聽到，於是走進店裡挑選幾張2000元刮刮樂，刮中一些小獎後打算離開時，當下又聽到鸚鵡講了「中獎！中獎！」，於是他們決定再買2張試試看。
台彩轉述，隔2天後這對夫妻又來到店裡，但卻一臉神秘兮兮的樣子拿了一張「2,000萬超級紅包」刮刮樂請店員協助確認是否有中獎，發現果真刮中100萬，夫妻倆開心的抱在一起大叫，並且認為真的是「有買有希望」。
----------------------------------------------------------------------

[71] 企業送禮、電商網購一波波　物流業者UPS延長春節前服務時間搶市

▲物流業者優比速（UPS）延長春節前服務時間，讓台灣企業有更充裕的時間處理緊急訂單。（圖／優比速提供）
記者林淑慧／台北報導
最近是農曆春節送禮潮，加上電商網購的採買需求，國內外物流業迎來產業旺季，美商優比速（UPS）今（20）日宣佈將於1月22日延長全台取件、電話預約時間和自行送件服務，因應農曆新年前的大量國際託運需求。
優比速（UPS）表示，因應農曆春節假期，UPS將於1月23日開放桃園泛太平洋轉運中心至晚上5點，讓客戶可自行送件至轉運中心。
UPS將於1月23日起暫停收取件服務，並於1月30日恢復正常營運。UPS各服務中心的延長服務時間及聯絡資訊如下，顧客可透過0800-365-868

▲ 新台幣兌美元今天開盤貶值2分。（圖／路透社）
記者李瑞瑾／台北報導
台北外匯市場新台幣兌美元匯率今（20）天開盤貶值2分、以29.98元兌1美元開出。
新台幣兌美元上周五（17日）收盤在29.96兌1美元；今天開盤以29.98元兌1美元開出、貶值2分。
新台幣開盤後在平盤附近震盪，截至上午9點06分，新台幣來到29.959元兌1美元，升值0.1分。
----------------------------------------------------------------------

[82] 台股封關日開紅！開盤上漲17點　站上12,100點

▲ 台股。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
中美協議樂觀情緒持續籠罩，美股四大指數續刷歷史新高。今（20）日來到台股封關日，開盤以12,107點開出、上漲17點。
台股持續震盪走高，截至上午9點03分，指數來到12,126點、上漲35點。
電子三雄中，台積電（2330）以334元開出、上漲0.3%；鴻海（2317）開在92.7元、上漲0.43%；股王大立光（3008）以5000元開出、上漲0.3%。
美股四大指數上周五（17日）續刷歷史新高。道瓊指數上漲50.46點，或0.17%，收29,348.1點；標普 500 指數上漲11.32點，或0.34%，收3,328.13點；那斯達克指數上漲31.81點，或0.34%，收9,388.94點；費城半導體指數上漲12.69點，或0.67%，收1,916.62點。
----------------------------------------------------------------------

[83] 投報率高雄最強！租金增值兩頭賺　包租公逢低搶進高雄CBD

▲高雄中正金融特區頂級商辦，投報率極高，吸引許多尋求穩定收益的包租公搶進，也為沉寂多年的高雄商辦市場點燃新戰火。（採訪撰稿／記者胡至欣；攝影剪輯／記者謝盛帆）
記者胡至欣／採訪報導
美中貿易戰尚未和解，加上政府釋出利多，台商回流熱度不減，投資金額超過8000億元，而人流、金流湧入，置產或設廠需求也跟著活絡，尤其以商辦大樓最為有感。
「以全台各大商業區來講，目前空置率是持續下降，甚至降到只有1%、2%的空置率，高雄中正金融特區，更是近20年都沒有全新的商辦規劃，另外租金價格也在往上浮升，一些大

記者楊絡懸／台北報導
中油19日宣布，20日凌晨0時起各式汽、柴油價格每公升價格各下降0.7元，調整後的參考零售價格分別為：92無鉛汽油每公升27.1元、95無鉛汽油每公升28.6元、98無鉛汽油每公升30.6元、超級柴油每公升25.0元。實際零售價格以各營業點公告為準。
台灣中油表示，受到美伊開戰導致荷姆茲海峽遭封鎖機率降低、中東地區緊繃情勢降溫等因素影響，帶動國際油價下跌。本週調價指標7D3B週均價為每桶64.66美元，較前週67.84美元下跌3.18美元，新台幣兌美元匯率升值0.109元，國內油價依公式計算跌幅為4.03％，92無鉛汽油及柴油均未超過亞鄰最低價，故汽、柴油應各調降0.7元，足額調整。
調價後的各式油品參考零售價格調幅及調整金額如上述，實際零售價格以各營業點公告為準。
▲晚點再加油！中油宣布油價調降了。（圖／記者張一中攝）
另一方面，台灣中油也考量春節是國人重要假期，為配合政府春節期間穩定物價政策，於春節期間連續二週（1月20日、1月27日），依「調降不調漲」的原則調整國內汽、柴油價格。
台灣中油強調，國內油價每週隨國際油價變動調整，有漲有跌，調幅大小完全取決於油價公式的計算結果，絕無「漲多跌少」或「只漲不跌」情事。國內所需的油源幾乎全從國外進口，來之不易，因此也呼籲請消費者體認全球油源蘊藏量有限，珍惜資源、節約用油，共同為地球環保盡一份心力。
----------------------------------------------------------------------

[93] 保險不是有保就必賠！　出國買對旅綜險有3大關鍵

▲春節連假即將到來，壽險公司提醒，民眾投保旅綜險要注意3大面向，才能玩得盡興又安心。（圖／取自免費圖庫Pixabay）
記者紀佳妘／台北報導
農曆春節即將到來，也是國人海外旅遊的高峰，壽險公司提醒，投保旅行綜合保險，應留意是否涵蓋旅行全程、保障是否完整、保額是否足夠、加強急難救助等3大面向，給自己多一份保障，才能玩得盡興又安心。
根據統計，截至2019年11月，國人出國人次相較2018年同期成長3.41%，尤其前往日本地區的人次將逾456萬，高居第一，顯示前往日本旅遊仍是多數國人首選。不過，旅行最怕突如其來的意外，像是班機延誤、行李遺失、護照被偷或氣候因素，導致班機無法起降等狀況，難免感到無力、掃興，壽險